Import Dependencies 

In [147]:
! pip install census
! pip install lxml
! pip install beautifulsoup4 html5lib
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install plotly==4.11.0
!pip install plotly-geo

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [148]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census API Key
c = Census(census_key, year=2017)

In [149]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
zipcode_census_data = c.acs5.get(("B01003_001E", "B17001_002E","B17001_003E","B17001_017E",
                          "B17001A_002E","B17001B_002E", "B17001C_002E" ,"B17001D_002E" ,
                          "B17001E_002E" ,"B17001G_002E" ,"B17001I_002E" ,"B17012_002E",
                          "B17012_009E","B17012_014E" ,"B23025_002E" ,"B23025_007E" ,
                          "B23025_004E" ,"B23025_005E"), {
                         'for': 'zip code tabulation area:*'})

# Convert to DataFrame
zipcode_census_pd = pd.DataFrame(zipcode_census_data)

#reorder names
zipcode_census_pd = zipcode_census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count","B17001_003E": "poverty_male",
                                                      "B17001_017E": "poverty_female","B17001A_002E": "poverty_white_alone",
                                                      "B17001B_002E": "poverty_black_alone",
                                                      "B17001C_002E": "population_american_indian_alone",
                                                      "B17001D_002E": "poverty_asian_alone",
                                                      "B17001E_002E": "poverty_native_hawaiian_alone",
                                                      "B17001G_002E": "poverty_two_or_more_races",
                                                      "B17001I_002E": "poverty_hispanic_origin",
                                                      "B17012_002E": "poverty_family",
                                                      "B17012_009E": "poverty_family_single_male",
                                                      "B17012_014E": "poverty_family_single_female",
                                                      "B23025_002E": "employment_labor_force",
                                                      "B23025_007E": "employment_not_labor_force"	,
                                                      "B23025_004E": "employment_employed",
                                                      "B23025_005E": "employment_unemployed",
                                      "zip code tabulation area": "Zipcode"})

zipcode_census_pd.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# Add in Poverty Rate (Poverty Count / Population)
zipcode_census_pd["Poverty Rate"] = 100 * \
    zipcode_census_pd["Poverty Count"].astype(
        int) / zipcode_census_pd["Population"].astype(int)

#Final DataFrame
# zipcode_census_pd = zipcode_census_pd[["Zipcode", "Population"]]

# # Visualize
print(len(zipcode_census_pd))
zipcode_census_pd.head()

33120


,Population,Poverty Count,poverty_male,poverty_female,poverty_white_alone,poverty_black_alone,population_american_indian_alone,poverty_asian_alone,poverty_native_hawaiian_alone,poverty_two_or_more_races,poverty_hispanic_origin,poverty_family,poverty_family_single_male,poverty_family_single_female,employment_labor_force,employment_not_labor_force,employment_employed,employment_unemployed,Zipcode,Poverty Rate
0,17599.0,11282.0,5379.0,5903.0,8765.0,84.0,10.0,0.0,0.0,97.0,11223.0,2607.0,201.0,1077.0,5953.0,8078.0,3499.0,2454.0,00601,64.105915
1,39209.0,20428.0,9961.0,10467.0,13813.0,557.0,0.0,0.0,0.0,5503.0,19144.0,4560.0,483.0,1762.0,14390.0,18022.0,11824.0,2538.0,00602,52.100283
2,50135.0,25176.0,11332.0,13844.0,17714.0,874.0,67.0,382.0,10.0,2402.0,24744.0,6405.0,566.0,3219.0,16044.0,25020.0,12389.0,3588.0,00603,50.216416
3,6304.0,4092.0,2057.0,2035.0,1819.0,63.0,0.0,0.0,0.0,109.0,4080.0,793.0,111.0,250.0,1707.0,3472.0,1503.0,204.0,00606,64.911168
4,27590.0,12553.0,5939.0,6614.0,7471.0,426.0,0.0,0.0,1.0,1063.0,12263.0,2716.0,294.0,1034.0,10048.0,12749.0,8574.0,1474.0,00610,45.498369


In [135]:
# Get California Data Only
    #Web scrape to get data for california zipcodes that includes county and whether 
tables = pd.read_html('Resources/zipcode_site.html')
zipcode_df = tables[2]


In [150]:
    #clean zipcode_df to get rid of p.o. boxes and rename columns
column_names = list(zipcode_df.iloc[0,:])
zipcode_df.columns = column_names
    #delete first row (redundant)
zipcode_df = zipcode_df.drop(zipcode_df.index[0])

In [151]:
#reset index 
zipcode_df=zipcode_df.reset_index()
#rename column for merge
zipcode_df = zipcode_df.rename(columns={"ZIP Code":"Zipcode"})

In [152]:
# zipcode_df.head()
zipcode_df['Zipcode']=zipcode_df['Zipcode'].str.slice(9)
zipcode_df

KeyError: 'Zipcode'

In [ ]:
set(zipcode_df['Type'])

In [ ]:
zipcode_df['Type'].value_counts()

In [ ]:
#drop all p.o. boxes 
zipcode_df = zipcode_df[~zipcode_df.Type.str.contains("P.O. Box|Unique")]
len(zipcode_df)

In [ ]:
#merge data frames to get california data only 
california_data = pd.merge(zipcode_census_pd, zipcode_df, on='Zipcode', how ='inner')
print(type(california_data))
print(len(california_data))
california_data.columns

In [ ]:
california_data['Type']
california_data

In [155]:
#Practice

california_data.to_csv("Resources/california_census_data.csv", encoding="utf-8", index=False)